In [ ]:
%pip install OSMPythonTools

In [1]:
from OSMPythonTools.overpass import Overpass
import pandas as pd
import os
import requests


In [3]:
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

OVERPASS_QUERY = """[out:json];
(
  node
    ["{}"="{}"]
    ({},{},{},{});
);
(._;>;);
out;"""


CSV_OUTPUT_FILENAME = "output.csv"
KEY = "railway"
VALUE = "station"

In [16]:
def extract_osm_data(bbox, key, value, url=OVERPASS_URL, query = OVERPASS_QUERY):
    map_features = []
    response = requests.post(url, data={'data': query.format(key, value, *bbox)})
    if response.status_code != 200:
        raise KeyError("Bad request, check OSM keys and values in the query")
    data = response.json()
    for elt in data.get('elements'):
        elt_dict = {
            "id": elt.get('id'),
            "lat": elt.get('lat'),
            "lon": elt.get('lon'),
        }
        elt_dict.update(elt.get('tags'))
        map_features.append(elt_dict)
    return map_features


In [11]:
def extract_osm_data_using_osm_tools(bbox, key, value, query=OVERPASS_QUERY):
    map_features=[]
    overpass = Overpass()
    result = overpass.query(query.format(key, value, *bbox))
    for elt in result.elements():
        elt_dict = {
            "id": elt.id(),
            "lat": elt.lat(),
            "lon": elt.lon(),
        }
        elt_dict.update(elt.tags())
        map_features.append(elt_dict)
    return map_features

def convert_map_features_to_csv(map_features, value):
    df = pd.DataFrame.from_dict(map_features)
    df.to_csv(os.path.join(os.getcwd(), value + '_' + CSV_OUTPUT_FILENAME))
    return df

#### using OSMPythonTools

In [13]:
#bbox = [42.189829808246266, -71.2912398492822, 42.48199986253678, -70.91697706096569]
bbox = [33.282091482933055, 8.070099366147417, 37.45555854420532, 11.51671581249958]
map_features = extract_osm_data_using_osm_tools(bbox, KEY, VALUE, query=OVERPASS_QUERY.replace('[out:json];', ''))
df = convert_map_features_to_csv(map_features, VALUE)
df.head()

[overpass] downloading data: [timeout:25][out:json];
(
  node
    ["railway"="station"]
    (33.282091482933055,8.070099366147417,37.45555854420532,11.51671581249958);
);
(._;>;);
out;


,id,lat,lon,addr:city,name,name:ar,name:fr,operator,operator:ar,operator:website,...,name:it,name:azb,name:ur,name:kab,population,name:az-Arab,name:fa,addr:street,addr:street:ar,subway
0,558822525,35.822860,10.641517,سوسة,سوسة باب الجديد,سوسة باب الجديد,Sousse Bab Djedid,الشركة الوطنية للسكك الحديدية التونسية,الشركة الوطنية للسكك الحديدية التونسية,http://www.sncft.com.tn/Ar/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,661316493,36.829219,10.190380,NaN,حي الخضراء,حي الخضراء,Cité El Khadra,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1089665035,36.815942,10.183215,NaN,محمد الخامس,محمد الخامس,Mohamed V,شركة النقل بتونس,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1094902351,36.820194,10.182199,NaN,فلسطين,فلسطين,Palestine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1094915522,36.823429,10.185575,NaN,الحدائق,الحدائق,Station Les Jardins,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Using requests

In [17]:
#bbox = [42.189829808246266, -71.2912398492822, 42.48199986253678, -70.91697706096569]
bbox = [33.282091482933055, 8.070099366147417, 37.45555854420532, 11.51671581249958]
map_features = extract_osm_data(bbox, KEY, VALUE, url=OVERPASS_URL, query = OVERPASS_QUERY)
df = convert_map_features_to_csv(map_features, VALUE)
df.head()

,id,lat,lon,addr:city,name,name:ar,name:fr,operator,operator:ar,operator:website,...,name:it,name:azb,name:ur,name:kab,population,name:az-Arab,name:fa,addr:street,addr:street:ar,subway
0,558822525,35.822860,10.641517,سوسة,سوسة باب الجديد,سوسة باب الجديد,Sousse Bab Djedid,الشركة الوطنية للسكك الحديدية التونسية,الشركة الوطنية للسكك الحديدية التونسية,http://www.sncft.com.tn/Ar/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,661316493,36.829219,10.190380,NaN,حي الخضراء,حي الخضراء,Cité El Khadra,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1089665035,36.815942,10.183215,NaN,محمد الخامس,محمد الخامس,Mohamed V,شركة النقل بتونس,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1094902351,36.820194,10.182199,NaN,فلسطين,فلسطين,Palestine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1094915522,36.823429,10.185575,NaN,الحدائق,الحدائق,Station Les Jardins,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
